In [17]:
import aiohttp
from string import Template
from typing import List

site_url: str = "https://www.welcometothejungle.com"
search_url: str = "/fr/jobs"
job_url: Template = Template("/jobs/$company/$job_name")
user_agents: List[str] = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.1.2 Safari/603.3.8",
    "Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36",
]
HEADERS = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }



In [25]:
from dataclasses import dataclass
from selectorlib import Extractor
import re
from dataclasses import dataclass, field


extractor = Extractor.from_yaml_file('rules_welcome.yaml')

def scrape_page(html: str):
    try:
        # Appliquer les règles définies dans le fichier YAML
        data = extractor.extract(html)
        return data
    except Exception as e:
        print(f"Error: {e}")
        return None

# creation des Entités qui structurent les données et permettent une conservation en base
@dataclass
class JobUrl:
    company: str
    job_name: str
    url: str = field(init=False)
    processed: bool = False

    def __post_init__(self):
        self.url = job_url.safe_substitute(company=self.company, job_name=self.job_name)
  

In [29]:
from typing import Dict, List
from itertools import cycle

user_agent_cycle = cycle(user_agents)

async def requests_jobs_summaries(query: str, cursor:int = 1) -> List[Dict]:
    """Effectue une requête pour lister les jobs.
        Dans le cas d'une pagination, on ajoute start_after= cursor du dernier job

    Args:
        query (str): requête dont les espaces sont remplacés par des +
    
    Returns:
        List[Dict]: liste des jobs
    """
    # query: str = re.sub(r"\s+", "", query)
    search_params: Dict = {
        'refinementList[offices.country_code][]': ['FR'],
        'query': [query],
        'page': [cursor]
    }
    
    user_agent = next(user_agent_cycle)
    headers = {"User-Agent": user_agent, "Accept": "application/json"}
    
    url: str = f"{site_url}{search_url}"
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(
                url=url, headers=headers, params=search_params
            ) as resp:
                if resp.status == 200:
                    html = await resp.text()
                    scrape = scrape_page(html)
                    print(scrape)
        except Exception as exc:
            print(exc)

jobs_ls: List = await requests_jobs_summaries(query = "Data engineer") 
print(jobs_ls)

{'jobs': None}
None


In [33]:
import nodriver

async def main():

    browser = await nodriver.start()
    page = await browser.get('https://www.nowsecure.nl')

    await page.save_screenshot()
    await page.get_content()
    await page.scroll_down(150)
    elems = await page.select_all('*[src]')

    for elem in elems:
        await elem.flash()

    page2 = await browser.get('https://twitter.com', new_tab=True)
    page3 = await browser.get('https://github.com/ultrafunkamsterdam/nodriver', new_window=True)

    for p in (page, page2, page3):
       await p.bring_to_front()
       await p.scroll_down(200)
       await p   # wait for events to be processed
       await p.reload()
       if p != page3:
           await p.close()

await main()

NotImplementedError: 

In [ ]:
# appel de la methode une 1ere fois
from pprint import pprint

job_urls_ls: List[JobUrl] = []
count: int = 0
cursor: str | None = None
query: str = "Data engineer"
has_more: bool = True

while has_more:
    # while count < 10:

    job_api_reponse: List[Dict] = await requests_jobs_summaries(
        query=query, cursor=cursor
    )
    scrap_result: Dict = await extract_summaries_infos(job_api_reponse)

    has_more: str = scrap_result.get("has_more")
    cursor: str = scrap_result.get("cursor")
    job_urls_ls.extend(scrap_result.get("job_urls_ls"))

    print(has_more)
    print(cursor)

    # simulation has_more=False apres 5 iterations
    count += 1
    if count > 2:
        has_more = False

print(len(job_urls_ls))
job_urls_ls

In [ ]:
import asyncio
from aiohttp import ClientSession


async def fetch_url(sem, session, url):
    """
    Récupère une URL en utilisant un sémaphore pour limiter les requêtes simultanées.
    """
    async with sem:  # Limiter le nombre de connexions simultanées
        try:
            async with session.get(url) as response:
                data = await response.text()
                print(f"Fetched {url} with status {response.status}")
                return data
        except Exception as e:
            print(f"Error fetching {url}: {e}")
            return None


async def fetch_all(urls: List[str], max_concurrent_requests: int=5):
    """
    Récupère plusieurs URLs en parallèle avec un maximum de requêtes simultanées.
    """
    # Initialisation du sémaphore
    sem = asyncio.Semaphore(max_concurrent_requests)
    
    # Création d'une session HTTP partagée
    async with ClientSession() as session:
        # Création des coroutines pour chaque URL
        tasks = [fetch_url(sem, session, url) for url in urls]
        
        # Exécution des tâches en parallèle
        results = await asyncio.gather(*tasks)
        return results


In [ ]:

urls = [f"{site_url}{job.url}" for job in  job_urls_ls]

results = await fetch_all(urls)
results

In [ ]:
from selectorlib import Extractor

extractor = Extractor.from_yaml_file('rules_muse.yaml')

def scrape_page(html: str):
    try:
        # Appliquer les règles définies dans le fichier YAML
        data = extractor.extract(html)
        return data
    except Exception as e:
        print(f"Error: {e}")
        return None

@dataclass 
class JobMuse:
    company_size: str
    company_domain: List[str]
    contenu: str

    def __post_init__(self):
        self.company_size = re.sub(r'Size\:\s*','',self.company_size)
        industries: str = re.sub(r'Industry:\s*','',self.company_domain)
        self.company_domain = industries.split(',')
    

In [ ]:
job_text_ls: List[JobMuse]=[]
for page in results[:5]:
    job_text_ls.append(JobMuse(**scrape_page(page)))

pprint(job_text_ls)